# Imports

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!export PYSPARK_PYTHON=python
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=08accc7747897de6ca05902272c070eb5eb8c9d02d8578259f5c957255f52882
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-upda

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math
from datetime import datetime

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Inicialización de los RDD

In [5]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

In [6]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)

apps_df = sqlContext.read.csv('googleplaystore.csv',
                              header=True,
                              inferSchema=True,
                              mode='DROPMALFORMED') \
                              .select("App", "Category", "Last Updated")
apps_rdd = apps_df.rdd

# Pregunta #29
[*ir al notebook general*](https://colab.research.google.com/drive/1wWj79hKJTtaauQYJndtlhfQ8KEiuKe8Q?usp=drive_link)

> *Calcular la antigüedad promedio de las app de categoría “Family”*

**Respuesta:** 2212 días

**Notas:**
- Uso la misma función para castear a datetime y para asignar algún valor a todas las filas inválidas para filtrar fácil
- El primer map y reduceByKey (después del filter) me quitan las apps duplicadas. Hago básicamente lo mismo (map a dupla nombre - campos que quiero y reduceByKey de agarrar uno de los 2) en el resto de los ejercicios que usan el dataset de apps. No tengo ningún criterio para seleccionar a uno de los duplicados, así que agarro uno cualquiera

In [13]:
def days_since(dstr):
  try:
    return (datetime.now() - datetime.strptime(dstr, "%B %d, %Y")).days
  except:
    return -1

reduced = apps_rdd \
  .filter(lambda x: "FAMILY" in x[1] and days_since(x[2]) != -1) \
  .map(lambda x: (x[0], x[2])) \
  .reduceByKey(lambda x,y: x) \
  .map(lambda x: (days_since(x[1]), 1)) \
  .reduce(lambda x, y: (x[0] + y[0], x[1] + y[1])) \

ans29 = reduced[0] / reduced[1]
print(f"Average days since last update in FAMILY category: {ans29:.0f} days")


Average days since last update in FAMILY category: 2212
